In [ ]:
# Prototype selecting primitive declarations with drop down
from IPython.display import clear_output, display_javascript, display, HTML
import ipywidgets as widgets
from ipywidgets import GridspecLayout

import os
import tempfile
import unittest
import filecmp
import sbol3
import paml
import tyto
import uml
from paml.execution_engine import ExecutionEngine

%load_ext autoreload
%autoreload 2

In [ ]:
#############################################
# set up the document
print('Setting up document')
doc = sbol3.Document()
sbol3.set_namespace('https://bbn.com/scratch/')

#############################################
# Import the primitive libraries
print('Importing libraries')
paml.import_library('liquid_handling')
print('... Imported liquid handling')
paml.import_library('plate_handling')
print('... Imported plate handling')
paml.import_library('spectrophotometry')
print('... Imported spectrophotometry')
paml.import_library('sample_arrays')
print('... Imported sample arrays')

primitives = { primitive.identity : primitive 
              for lib, lib_doc in paml.loaded_libraries.items()
              for primitive in lib_doc.objects }

In [ ]:
hidden_html = HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('.cm-comment:contains(@hidden)').closest('div.input').hide();
 } else {
 $('.cm-comment:contains(@hidden)').closest('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script><a href="javascript:code_toggle()">show</a>.''')

def make_and_exec_new_cell(text : str):
    display_javascript("""var t_cell = Jupyter.notebook.insert_cell_below()
    t_cell.set_text('# @hidden\\n{}\\n{}');
    var t_index = IPython.notebook.get_cells().indexOf(t_cell);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.get_cell(t_index).execute();
    Jupyter.notebook.execute_cells(t_index);""".format(display(hidden_html), text.replace('\n','\\n')), raw=True)

def make_new_cell(text : str):
    display_javascript("""var t_cell = Jupyter.notebook.insert_cell_below()
    t_cell.set_text('{}');
    var t_index = IPython.notebook.get_cells().indexOf(t_cell);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    Jupyter.notebook.execute_cells(t_index);""".format(text.replace('\n','\\n').replace('\'', "\\'")), raw=True)



In [ ]:
layout_hidden  = widgets.Layout(visibility = 'hidden')
layout_visible = widgets.Layout(visibility = 'visible')

def renderUI():
    make_and_exec_new_cell("ui")
    
def new_protocol_event(b):
    renderUI()
    make_new_cell(f"{paml.Protocol.template()}")
    
def create_primitive_event(b):
    #renderUI()
    #make_new_cell(primitive_template)
    new_primitive_drop_down.layout = layout_visible
    new_primitive_drop_down_ok.layout = layout_visible

def new_primitive_event(b):
    renderUI()
    #print(primitive.template())
    make_new_cell(primitive.template())
    #make_new_cell("foo")
    #new_primitive_drop_down.layout = layout_visible
    #new_primitive_drop_down_ok.layout = layout_visible


#protocol_template = f"protocol"
#primitive_template = f"primitive"
#primitives = ["a", "b", "c"]
primitive_keys = list(primitives.keys())

new_protocol_button = widgets.Button(description="New Protocol", 
                                     tooltip="Create a template for a new protocol.")
new_primitive_button = widgets.Button(description="New Primitive", 
                                     tooltip="Select a template for a new primitive.")
new_primitive_drop_down = widgets.Dropdown(
    options=primitive_keys,
    value=primitive_keys[0],
    description='Primitive:',
    layout = layout_hidden
)
new_primitive_drop_down_ok = widgets.Button(description="Add", 
                                     tooltip="Create a template for a new primitive.",
                                           layout=layout_hidden)

new_protocol_button.on_click(new_protocol_event)
new_primitive_button.on_click(create_primitive_event)
new_primitive_drop_down_ok.on_click(new_primitive_event)

ui = GridspecLayout(1, 4)
ui[0, 0] = new_protocol_button
ui[0, 1] = new_primitive_button
ui[0, 2] = new_primitive_drop_down
ui[0, 3] = new_primitive_drop_down_ok
ui